# Use AutoEncoding with force_torque_sensor__CPU


--------------------------------------------------------------------

Datasets paper: https://arxiv.org/pdf/1807.06749.pdf

Download: https://ibm.ent.box.com/s/vw4y576xlz6ujblpl3gz9c5ttu51qc18

Modified by github : "Variational Autoencoder & Conditional Variational Autoenoder on MNIST"

In [1]:
import numpy as np 

sides_3 = np.load('../data/force_torque_sensor/Dataset/3_sides/Data/data.npy')
# sides_4 = np.load('../data/force_torque_sensor/Dataset/4_sides/Data/data.npy')
# sides_5 = np.load('../data/force_torque_sensor/Dataset/5_sides/Data/data.npy')
# sides_6 = np.load('../data/force_torque_sensor/Dataset/6_sides/Data/data.npy')
# sides_200 = np.load('../data/force_torque_sensor/Dataset/200_sides/Data/data.npy')


import numpy as np 
from collections import namedtuple
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.plotly as py
import plotly


/home/jim/anaconda2/envs/clustering/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# import matplotlib as mpl
# import numpy as np
# import matplotlib.pyplot as plt

# cmap=plt.cm.get_cmap(plt.cm.viridis,143)


# # plt.rcParams['font.size'] = 11.
# # plt.rcParams['font.family'] = 'Comic Sans MS'
# # plt.rcParams['axes.labelsize'] = 15.
# # plt.rcParams['xtick.labelsize'] = 10.
# # plt.rcParams['ytick.labelsize'] = 10.

# plt.figure(figsize=(30,10))



# sides = sides_3
# print(len(sides))

# Force_x = sides[:,0] 
# Force_y = sides[:,1]
# Force_z = sides[:,2]
# Moment_x = sides[:,3]
# Moment_y = sides[:,4]
# Moment_z = sides[:,5]
# Peg_Position_x = sides[:,6]
# Peg_Position_y = sides[:,7]
# Peg_Position_z = sides[:,8]
# Angle = sides[:,9]
# Time = sides[:,10]
# Counter = sides[:,11]


# plt.plot(Time, Force_x, marker='s', linestyle='-', markersize=2, linewidth=1, label='Force_x')
# plt.plot(Time, Force_y, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_y')
# plt.plot(Time, Force_z, marker='o', linestyle='-', markersize=2, linewidth=1, label='Force_z')

# plt.plot(Time, Moment_x, marker='o', linestyle='-', label='Moment_x')
# plt.plot(Time, Moment_y, marker='o', linestyle='-', label='Moment_y')
# plt.plot(Time, Moment_z, marker='o', linestyle='-', label='Moment_z')

# plt.plot(Time, Peg_Position_x, marker='o', linestyle='-', label='Peg_Position_x')
# plt.plot(Time, Peg_Position_y, marker='o', linestyle='-', label='Peg_Position_y')
# plt.plot(Time, Peg_Position_z, marker='o', linestyle='-', label='Peg_Position_z')

# plt.plot(Time, Angle, marker='o', linestyle='-', label='Angle')
# plt.plot(Time, Counter, marker='o', linestyle='-', label='Counter')


# plt.xlabel('Times')
# plt.ylabel('Signals')
# plt.title('3 Sileds force torque signals')
# plt.legend(loc='lower right')
# plt.show()

In [3]:
import os
import time
import torch
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from collections import defaultdict
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler



from models import VAE

import numpy as np




## Normalize

In [4]:
def feature_normalize(data):
    mu = np.mean(data,axis=0)
    std = np.std(data,axis=0)
    return (data - mu)/std

def un_normalize(normalized_data, input_data):
    mu = np.mean(input_data,axis=0)
    std = np.std(input_data,axis=0)
    return normalized_data*std+mu
    

In [5]:
train_data = feature_normalize(sides_3)
test_data =feature_normalize(sides_3)

# number of subprocesses to use for data loading
num_workers = 4
# how many samples per batch to load
batch_size = 8
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [6]:
latent_size = 2

import torch.utils.data
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers,
                                           sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers,
                                           sampler=valid_sampler)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

print (len(train_data))
print (len(train_loader))
print (len(valid_loader))
# print(len(train_data.size(0)))

1585353
158536
39634


In [7]:
vae = VAE(latent_size=latent_size).float()

optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)

logs = defaultdict(list)
print(vae)

# use_cuda = torch.cuda.is_available
# if use_cuda:
#     vae.cuda()

VAE(
  (encoder): Encoder(
    (fc1): Linear(in_features=12, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=8, bias=True)
    (linear_means): Linear(in_features=8, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=8, out_features=2, bias=True)
  )
  (decoder): Decoder(
    (fc3): Linear(in_features=2, out_features=8, bias=True)
    (fc4): Linear(in_features=8, out_features=32, bias=True)
    (fc5): Linear(in_features=32, out_features=12, bias=True)
  )
)


In [8]:
criterion = torch.nn.MSELoss()

def loss_fn(recon_x, x, mean, log_var):
    BCE = criterion(recon_x.view(-1, 12), x.view(-1, 12))
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

    return (BCE + KLD) / x.size(0)

In [ ]:
epochs = 20
print_every = 100

for epoch in range(epochs):
    
    for iteration, d in enumerate(train_loader):
#         if use_cuda:
#             x = d.cuda().float()
#         else:
        x = d.float()
        recon_x, mean, log_var, z = vae(x)
        loss = loss_fn(recon_x, x, mean, log_var).cuda()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        logs['loss'].append(loss.item())

        if iteration % print_every == 0 or iteration == len(train_loader)-1:
            correct = 0
            total = 0
            for valid in valid_loader:
#                 if use_cuda:
#                     input_valid = valid.cuda().float()
#                 else:
                input_valid = valid.float()

                valid_recon_x, valid_mean, valid_log_var, valid_z = vae(input_valid)
                valid_loss = loss_fn(valid_recon_x, input_valid,
                                     valid_mean, valid_log_var)
                total +=1

                if valid_loss> -0.015 and valid_loss < 0.015:
                    correct += 1

            valid_acc = correct / total  
            
            print("Epoch {:02d}/{:02d} Batch {:04d}/{:d}, Train_loss {:9.4f}, Valid_accuary{:9.4f}".format(
                epoch, epochs, iteration, len(train_loader)-1, loss.item(), valid_acc))

    torch.save(cpu_vae, 'cpu_vae.pkl')  
        
    if loss < 0.001:
        break


In [ ]:
params = list(vae.named_parameters())
print(params)

In [ ]:
torch.save(vae, 'vae.pkl')


In [ ]:
import torch
vae = torch.load('vae.pkl')
print(vae)